# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe9c09d3820>
├── 'a' --> tensor([[-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302]])
└── 'x' --> <FastTreeValue 0x7fe9c09d3130>
    └── 'c' --> tensor([[-0.1852,  1.9320,  0.4704, -0.1873],
                        [ 1.0247,  0.8216, -0.2618,  0.5323],
                        [-0.3700,  1.2967, -1.4689,  1.0519]])

In [4]:
t.a

tensor([[-0.3428, -0.0762,  2.2447],
        [ 0.1340, -0.2880, -0.7302]])

In [5]:
%timeit t.a

78.6 ns ± 0.545 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe9c09d3820>
├── 'a' --> tensor([[-1.0795, -0.7507, -0.2101],
│                   [-0.0722, -0.1330,  0.4062]])
└── 'x' --> <FastTreeValue 0x7fe9c09d3130>
    └── 'c' --> tensor([[-0.1852,  1.9320,  0.4704, -0.1873],
                        [ 1.0247,  0.8216, -0.2618,  0.5323],
                        [-0.3700,  1.2967, -1.4689,  1.0519]])

In [7]:
%timeit t.a = new_value

78.4 ns ± 0.934 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302]]),
    x: Batch(
           c: tensor([[-0.1852,  1.9320,  0.4704, -0.1873],
                      [ 1.0247,  0.8216, -0.2618,  0.5323],
                      [-0.3700,  1.2967, -1.4689,  1.0519]]),
       ),
)

In [10]:
b.a

tensor([[-0.3428, -0.0762,  2.2447],
        [ 0.1340, -0.2880, -0.7302]])

In [11]:
%timeit b.a

76.7 ns ± 0.518 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4258,  0.8455,  1.1242],
               [-0.8521, -0.1673,  1.3374]]),
    x: Batch(
           c: tensor([[-0.1852,  1.9320,  0.4704, -0.1873],
                      [ 1.0247,  0.8216, -0.2618,  0.5323],
                      [-0.3700,  1.2967, -1.4689,  1.0519]]),
       ),
)

In [13]:
%timeit b.a = new_value

663 ns ± 6.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.11 µs ± 20.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

322 µs ± 52.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

281 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe993ec1520>
├── 'a' --> tensor([[[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]],
│           
│                   [[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]],
│           
│                   [[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]],
│           
│                   [[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]],
│           
│                   [[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]],
│           
│                   [[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]],
│           
│                   [[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]],
│           
│                   [[-0.3428, -0.0762,  2.2447],
│                    [ 0.1340, -0.2880, -0.7302]]])
└── 'x' --> <FastTreeValue 0x7fe993ec1d30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

51.9 µs ± 2.81 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe9146a0160>
├── 'a' --> tensor([[-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302],
│                   [-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302],
│                   [-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302],
│                   [-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302],
│                   [-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302],
│                   [-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302],
│                   [-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302],
│                   [-0.3428, -0.0762,  2.2447],
│                   [ 0.1340, -0.2880, -0.7302]])
└── 'x' --> <FastTreeValue 0x7fe9146a0610>
    └── 'c' --> tensor([[-0.1852,  1.9320,  0.4704, -0.1873],
                        [ 1.0247,  0.8216, -0.2618,  0.5323],
                 

In [23]:
%timeit t_cat(trees)

43 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.6 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]],
       
               [[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]],
       
               [[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]],
       
               [[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]],
       
               [[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]],
       
               [[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]],
       
               [[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]],
       
               [[-0.3428, -0.0762,  2.2447],
                [ 0.1340, -0.2880, -0.7302]]]),
    x: Batch(
           c: tensor([[[-0.1852,  1.9320,  0.4704, -0.1873],
                       [ 1.0247,  0.8216, -0.2618,  0.5323],
                       [-0.3700,  1.2967, -1.4689,  1.0519]],
         

In [26]:
%timeit Batch.stack(batches)

113 µs ± 4.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302],
               [-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302],
               [-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302],
               [-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302],
               [-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302],
               [-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302],
               [-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302],
               [-0.3428, -0.0762,  2.2447],
               [ 0.1340, -0.2880, -0.7302]]),
    x: Batch(
           c: tensor([[-0.1852,  1.9320,  0.4704, -0.1873],
                      [ 1.0247,  0.8216, -0.2618,  0.5323],
                      [-0.3700,  1.2967, -1.4689,  1.0519],
                      [-0.1852,  1.9320,  0.4704, -0.1873],
                      [ 1.0247,  

In [28]:
%timeit Batch.cat(batches)

199 µs ± 5.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

658 µs ± 60.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
